# Dataset Research

> We will use SQL queries just for experience.

In [1]:
import psycopg
import pandas as pd

In [2]:
with open("../../../JupyterFiles/dbpass.txt", "r") as pwrdfile:
    usrnm = pwrdfile.readline().strip()
    pwrd = pwrdfile.readline().strip()
    dbnm = pwrdfile.readline().strip()

In [34]:
with psycopg.connect(dbname=dbnm, user=usrnm, password=pwrd) as conn:
    with conn.cursor() as cur:

        cur.execute("SET search_path TO mvp_schema")

        cur.execute("""
            SELECT * FROM mvp_dataset md
            JOIN series_list sl on sl.series_id = md.series_id
            """)
        
        dataset = cur.fetchall()

        cur.execute("""SELECT attname AS column_name
FROM pg_catalog.pg_attribute
JOIN pg_catalog.pg_class ON pg_class.oid = pg_attribute.attrelid
JOIN pg_catalog.pg_namespace ON pg_namespace.oid = pg_class.relnamespace
WHERE pg_namespace.nspname = 'mvp_schema' AND pg_class.relname = 'mvp_dataset' AND pg_attribute.attnum > 0 AND NOT pg_attribute.attisdropped
UNION ALL
SELECT attname AS column_name
FROM pg_catalog.pg_attribute
JOIN pg_catalog.pg_class ON pg_class.oid = pg_attribute.attrelid
JOIN pg_catalog.pg_namespace ON pg_namespace.oid = pg_class.relnamespace
WHERE pg_namespace.nspname = 'mvp_schema' AND pg_class.relname = 'series_list' AND pg_attribute.attnum > 0 AND NOT pg_attribute.attisdropped;
""")

        colnames = cur.fetchall()

        df = pd.DataFrame(data=dataset)

In [35]:
coln = [tup[0] for tup in colnames]
df.columns = coln

In [37]:
df.head()

,country_code,series_id,2000,2001,2002,2003,2004,2005,2006,2007,...,2012,2013,2014,2015,2016,2017,2018,2019,series_id,series_name
0,ARG,1,19.366,18.983,18.756,18.453,18.352,18.353,18.194,18.005,...,17.729,17.632,17.504,17.346,16.824,16.206,15.187,14.783,1,"Birth rate, crude (per 1,000 people)"
1,AUT,1,9.8,9.4,9.7,9.5,9.7,9.5,9.4,9.2,...,9.4,9.4,9.6,9.8,10.0,10.0,9.7,9.6,1,"Birth rate, crude (per 1,000 people)"
2,BLR,1,9.4,9.2,8.9,9.0,9.1,9.3,9.9,10.7,...,12.2,12.5,12.5,12.5,12.4,10.8,9.9,9.3,1,"Birth rate, crude (per 1,000 people)"
3,BEL,1,11.4,11.2,10.9,11.0,11.3,11.4,11.6,11.7,...,11.5,11.3,11.2,10.8,10.8,10.5,10.4,10.2,1,"Birth rate, crude (per 1,000 people)"
4,CRI,1,19.788,19.104,18.12,17.656,17.228,16.99,16.797,16.823,...,15.483,15.093,14.852,14.522,14.145,13.926,13.562,12.847,1,"Birth rate, crude (per 1,000 people)"


In [42]:
df.drop('series_id',axis=1,inplace=True)

In [43]:
df.head()

,country_code,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,series_name
0,ARG,19.366,18.983,18.756,18.453,18.352,18.353,18.194,18.005,17.812,...,17.806,17.729,17.632,17.504,17.346,16.824,16.206,15.187,14.783,"Birth rate, crude (per 1,000 people)"
1,AUT,9.8,9.4,9.7,9.5,9.7,9.5,9.4,9.2,9.3,...,9.3,9.4,9.4,9.6,9.8,10.0,10.0,9.7,9.6,"Birth rate, crude (per 1,000 people)"
2,BLR,9.4,9.2,8.9,9.0,9.1,9.3,9.9,10.7,11.1,...,11.5,12.2,12.5,12.5,12.5,12.4,10.8,9.9,9.3,"Birth rate, crude (per 1,000 people)"
3,BEL,11.4,11.2,10.9,11.0,11.3,11.4,11.6,11.7,11.9,...,11.7,11.5,11.3,11.2,10.8,10.8,10.5,10.4,10.2,"Birth rate, crude (per 1,000 people)"
4,CRI,19.788,19.104,18.12,17.656,17.228,16.99,16.797,16.823,16.899,...,15.772,15.483,15.093,14.852,14.522,14.145,13.926,13.562,12.847,"Birth rate, crude (per 1,000 people)"


So. The main question is do we have any correlations?

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
with psycopg.connect(dbname=dbnm, user=usrnm, password=pwrd) as conn:
    with conn.cursor() as cur:

        cur.execute("SET search_path TO mvp_schema")

        cur.execute("""
            SELECT * FROM series_list
            """)
        
        dataset = cur.fetchall()

        cur.execute("""SELECT attname AS column_name
FROM pg_catalog.pg_attribute
JOIN pg_catalog.pg_class ON pg_class.oid = pg_attribute.attrelid
JOIN pg_catalog.pg_namespace ON pg_namespace.oid = pg_class.relnamespace
WHERE pg_namespace.nspname = 'mvp_schema' AND pg_class.relname = 'series_list' AND pg_attribute.attnum > 0 AND NOT pg_attribute.attisdropped;""")

        colnames = cur.fetchall()

        df = pd.DataFrame(data=dataset)

NameError: name 'plt' is not defined